# TensorFlow 

## Introduction

In [1]:
# import and suppres warnings 
# ... (if you don't want to suppress them, you need to match the numpy version)
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from IPython.display import Markdown, display
import tensorflow as tf
tf.__version__

from tensorflow.python.client import device_lib
print(str(device_lib.list_local_devices()))
# display(Markdown(str(device_lib.list_local_devices())))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14063579232049121562
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17776696459528871787
physical_device_desc: "device: XLA_CPU device"
]


In [ ]:
import tensorflow as tf
with tf.compat.v1.Session() as sess:
    # verify that the math works
    a = tf.constant(50)
    b = tf.constant(51)
    print("a + b = {0}".format(sess.run(a + b)))
    
    sess.close()

a + b = 101


**Eager Execution**

Eager mode evaluates operations immediatley and return concrete values immediately. 

**Graph Execution**

Graph mode was TensorFlow's default execution mode. In graph mode operations only produce a symbolic graph which doesn't get executed until run within the context of a tf.Session(). This style of coding is less inutitive and has more boilerplate, however it can lead to performance optimizations and is particularly suited for distributing training across multiple devices. We recommend using delayed execution for performance sensitive production code. 

In [ ]:
import tensorflow.compat.v1 as tf
print(tf.__version__)

2.2.0


## Graph Execution

### Adding Two Tensors 

#### Build the Graph

Unlike eager mode, no concrete value will be returned yet. Just a name, shape and type are printed. Behind the scenes a directed graph is being created.

In [ ]:
tf.compat.v1.disable_eager_execution()
a = tf.constant(value = [5, 3, 8], dtype = tf.int32)
b = tf.constant(value = [3, -1, 2], dtype = tf.int32)
c = tf.add(a, b)
print(c)

Tensor("Add_1:0", shape=(3,), dtype=int32)


Let's compare to numpy

In [ ]:
import numpy as np

a1 = np.array([5, 3, 8])
b1 = np.array([3, -1, 2])
c1 = a1 + b1
print(c1)

[ 8  2 10]


Numpy immediatelly executed. In lazy mode - TF did not. That is the difference between lazy and eager execution.

#### Run the Graph

A graph can be executed in the context of a `tf.compat.v1.Session()`. Think of a session as the bridge between the front-end Python API and the back-end C++ execution engine. 

Within a session, passing a tensor operation to `run()` will cause Tensorflow to execute all upstream operations in the graph required to calculate that value.

In [ ]:
sess = tf.compat.v1.Session()
print(sess.run(c))

[ 8  2 10]


#### Parameterizing the Graph 

What if values of `a` and `b` keep changing? How can you parameterize them so they can be fed in at runtime? 

*Step 1: Define Placeholders*

Define `a` and `b` using `tf.placeholder()`. You'll need to specify the data type of the placeholder, and optionally a tensor shape.

*Step 2: Provide feed_dict*

Now when invoking `run()` within the `tf.Session()`, in addition to providing a tensor operation to evaluate, you also provide a dictionary whose keys are the names of the placeholders. 

In [ ]:
a = tf.compat.v1.placeholder(dtype = tf.int32, shape = [None])  
b = tf.compat.v1.placeholder(dtype = tf.int32, shape = [None])
c = tf.add(x = a, y = b)

with tf.compat.v1.Session() as sess:
    result = sess.run(fetches = c, feed_dict = {
        a: [3, 4, 5],
        b: [-1, 2, 3]
    })
    print(result)

[2 6 8]


## Graph execution with TF2 using functional api

The way you create a graph in TensorFlow is to use tf.function, either as a direct call or as a decorator.

In [ ]:
# restart notebook before running
import tensorflow as tf

tf.compat.v1.enable_eager_execution()

# Define a Python function
def function_to_get_faster(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# Create a `Function` object that contains a graph
a_function_that_uses_a_graph = tf.function(function_to_get_faster)

# Make some tensors
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

# It just works!
a_function_that_uses_a_graph(x1, y1, b1).numpy()

array([[12.]], dtype=float32)

In [ ]:
# Use the decorator
@tf.function
def function_to_get_faster(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

function_to_get_faster(x1, y1, b1).numpy()

array([[12.]], dtype=float32)

The pattern to follow is to define the training step function, that's the most computationally intensive function, and decorate it with @tf.function: only use tf.function to decorate high-level computations - for example, one step of training, or the forward pass of your model.

https://stackoverflow.com/a/55279561/1964707

## Eager Execution

In [ ]:
# restart notebook before running
import os
import tensorflow as tf

tf.executing_eagerly()

True

Now you can run TensorFlow operations and the results will return immediately (no sessions required):

In [ ]:
x = [[2.]]
m = tf.matmul(x, x)
print("Result: {}".format(m))

Result: [[4.]]


Enabling eager execution changes how TensorFlow operations behave—now they immediately evaluate and return their values to Python. tf.Tensor objects reference concrete values instead of symbolic handles to nodes in a computational graph. Since there isn't a computational graph to build and run later in a session, it's easy to inspect results using print() or a debugger. Evaluating, printing, and checking tensor values does not break the flow for computing gradients.

## Tensors

### Initialize

In [ ]:
a = tf.constant([[1.0, 2.0]])
b = tf.constant([[4.0, 3.0]])

### Operator overloading is supported

In [ ]:
print(a * b)
print(a + b)
print(a - b)
print(a / b)
print(a @ tf.transpose(b)) # matrix multiplication

tf.Tensor([[4. 6.]], shape=(1, 2), dtype=float32)
tf.Tensor([[5. 5.]], shape=(1, 2), dtype=float32)
tf.Tensor([[-3. -1.]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.25      0.6666667]], shape=(1, 2), dtype=float32)
tf.Tensor([[10.]], shape=(1, 1), dtype=float32)


### Stacking

In [ ]:
stacked1 = tf.stack([b, b])
print(stacked1)

stacked2 = tf.stack([stacked1, stacked1])
print(stacked2)

tf.Tensor(
[[[4. 3.]]

 [[4. 3.]]], shape=(2, 1, 2), dtype=float32)
tf.Tensor(
[[[[4. 3.]]

  [[4. 3.]]]


 [[[4. 3.]]

  [[4. 3.]]]], shape=(2, 2, 1, 2), dtype=float32)


### Slicing

This operation extracts a slice of size size from a tensor input_ starting at the location specified by begin

In [ ]:
# tf.slice(
#     input_, begin, size, name=None
# )

# non_sliced = tf.constant([1, 2, 3])
# sliced = non_sliced[:] # get all
# sliced = non_sliced[:2] # 0th to 2nd element
# sliced = non_sliced[0:2] # 0th to 2nd again
# sliced = non_sliced[1:2] # 1st to 2nd

non_sliced = tf.constant([[1, 2, 3],[4, 5, 6]])
# sliced = non_sliced[0,] # take 1st subarray
# sliced = non_sliced[1,] # take 2nd subarray
# sliced = non_sliced[2,] # slice index 2 of dimension 0 out of bounds.
# sliced = non_sliced[:,2] # take the 3rd element from all subarrays
sliced = non_sliced[1:2,1:2] # take the 2nd element from 1st and 2nd subarrays
# non_sliced[(slicing the 1st dimention),(slicing the 2nd dimension)]
print(sliced)

tf.Tensor([[5]], shape=(1, 1), dtype=int32)


### Reshaping

Given tensor, this operation returns a new tf.Tensor that has the same values as tensor in the same order, except with a new shape given by shape.

In [ ]:
t1 = [[1, 2, 3],[4, 5, 6]]
print(type(t1)) # python list
print(tf.shape(t1))

t2 = tf.reshape(t1, [6])
print(t2)

<class 'list'>
tf.Tensor([2 3], shape=(2,), dtype=int32)
tf.Tensor([1 2 3 4 5 6], shape=(6,), dtype=int32)


The tf.reshape does not change the order of or the total number of elements in the tensor, and so it can reuse the underlying data buffer. This makes it a fast operation independent of how big of a tensor it is operating on.

In [ ]:
# tf.reshape([1, 2, 3], [2, 2])

InvalidArgumentError: Input to reshape is a tensor with 3 values, but the requested shape has 4 [Op:Reshape]

<h2> Heron's Formula in TensorFlow </h2>

The area of triangle whose three sides are $(a, b, c)$ is $\sqrt{s(s-a)(s-b)(s-c)}$ where $s=\frac{a+b+c}{2}$ 

In [ ]:
def compute_area(sides):
    # slice the input to get the sides
    a = sides[:,0]  # 5.0, 2.3
    b = sides[:,1]  # 3.0, 4.1
    c = sides[:,2]  # 7.1, 4.8

    # Heron's formula
    s = (a + b + c) * 0.5   # (a + b) is a short-cut to tf.add(a, b)
    areasq = s * (s - a) * (s - b) * (s - c) # (a * b) is a short-cut to tf.multiply(a, b), not tf.matmul(a, b)
    return tf.sqrt(areasq)


# pass in two triangles
area = compute_area(tf.constant([
  [5.0, 3.0, 7.1],
  [2.3, 4.1, 4.8]
]))
print(area.numpy())

[6.278497 4.709139]


## Debugging

### Error messages

In [ ]:
def method(data):
    a = data[:,0:2]
    b = data[:,1]
    c = a + b
    print(b.get_shape())
    return tf.matmul(c, tf.transpose(c))

print(method(tf.constant([
    [5.0, 3.0, 7.1],
    [2.3, 4.1, 4.8],
    [2.3, 4.1, 4.8]
])))

InvalidArgumentError: ignored

Incompatible shapes: [3,2] vs. [3] [Op:AddV2]

### Method isolation

### Made up data

### Common problems